# Working with CBECS 2018 Microdata in Python

# Background

We assume you're here because you're already familiar with [CBECS](https://www.eia.gov/consumption/commercial/about.php). The 2018 Commercial Buildings Energy Consumption Survey (CBECS) consists of Commercial Building characteristics, energy consumption and expenditures data, representing almost 6 million commercial buildings across the U.S.

The EIA already provides [22 tables](https://www.eia.gov/consumption/commercial/data/2018/index.php?view=characteristics) with detailed crosstabulations of 2018 CBECS building characteristics data, and [48 tables](https://www.eia.gov/consumption/commercial/data/2018/index.php?view=consumption) with cross tabulations of CBECS consumption, expenditures, and end-use data.

However, for those who are comfortable with data analysis, EIA also provides the CBECS "[microdata](https://www.eia.gov/consumption/commercial/data/2018/index.php?view=microdata)", and accompanying codebook. This microdata (available as CSV):
>contains 6,436 records. They represent commercial buildings from the 50 states and the District of Columbia. Each record **corresponds to a single response from an in-scope building** in the sample. The sample represents an estimated 5.9 million total buildings in the United States.

Put another way, each row of the CSV represents one commercial building. So the microdata allows us to dive deeper into the data **at the facility level**, instead of the aggregations that EIA provides in the Building Characteristics and Energy Consumption/Expenditures tables.

With it we can, for example, calculate the total Major Fuel (e.g. electricity, fuel oil, liquefied petroleum gases, natural gas, district steam, district hot water, and district chilled water) consumption for each Principal Building Activity (e.g. office, hospital, etc).

But, perhaps more interestingly, we can also look at the **descriptive statistics of the samples**, as opposed to just averages or aggregations, to (hopefully) develop a more nuanced and comprehensive understanding of the Commercial Sectors' energy consumption habits.

# A Notable Omission 🐍
There is a [CBECS User's Guide to the Public Use Microdata File](https://www.eia.gov/consumption/commercial/data/2018/pdf/Users%20Guide%20to%20the%202018%20CBECS%20Public%20Use%20Microdata%20File.pdf), with a set of examples in Excel, SAS and R. 

However, considering it's prominence as a lingua franca in the data world, a notable omission is any relevant examples for Python. We'll re-create the 6 Excel examples from the CBECS microdata User's Guide using Python.

We've built some tools to help you hit the ground running with CBECS microdata in Python. The following assumes familiarity with the Python package `pandas`, and it's primary data types, `DataFrames` and `Series`.

# Getting Started

Let's load some helpers. We assume you've already downloaded the microdata and codebook, and placed them in a `data` directory relative to this notebook.

In [12]:
from pycbecs.io import Cbecs2018DataHandler # For loading data
from pycbecs.codes import (
    CodeHeaders as cdh, # Enums for accessing data
    MicroDataHeaders2018 as mdh # Enums for accessing data
)

In [13]:
handler = Cbecs2018DataHandler()
codebook = handler.load_codebook("data/2018microdata_codebook.xlsx")
microdata = handler.load_microdata("data/cbecs2018_final_public.csv")

In [14]:
microdata

,PUBID,REGION,CENDIV,PBA,PUBCLIM,SQFT,SQFTC,WLCNS,RFCNS,RFCOOL,...,ZMFBTU,ZMFEXP,ZELCNS,ZELEXP,ZNGCNS,ZNGEXP,ZFKCNS,ZFKEXP,ZDHBTU,ZDHEXP
0,1,3,5,2,3,210000,8,1,4,2,...,0,0,0,0,9,9,1,1,0,0
1,2,4,9,2,4,28000,5,1,6,1,...,0,0,0,0,0,0,9,9,9,9
2,3,3,5,8,4,2100,2,1,4,2,...,0,0,0,0,9,9,9,9,9,9
3,4,3,7,5,5,240000,8,2,6,1,...,0,0,0,0,1,1,9,9,9,9
4,5,1,2,5,3,295000,8,3,6,2,...,0,0,0,0,0,0,9,9,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6431,6432,4,8,14,2,130000,7,1,1,2,...,0,0,0,0,0,0,9,9,9,9
6432,6433,3,7,1,4,1050,2,1,5,2,...,9,9,9,9,9,9,9,9,9,9
6433,6434,3,5,2,4,122000,7,2,6,1,...,1,1,1,1,9,9,9,9,9,9
6434,6435,3,5,23,4,15000,4,3,5,1,...,2,2,2,2,2,2,9,9,9,9


At first glance, the microdata can be quite overwhelming (it's 1249 columns). Luckily, EIA provides a "codebook" to help understand nomenclature. For example, what does the column `CENDIV` (above) represent? The codebook provides some "metadata" descriptions for each column. We've represented the codebook as a pandas dataframe, where each column header from the microdata is a row in the codebook dataframe.

In [15]:
codebook

,var_order,var_type,label,codes,question_text
var_name,,,,,
PUBID,1,Char,Public use file building identifier,00001 – 06436,NaN
REGION,2,Num,Census region,1=Northeast\n2=Midwest\n3=South\n4=West,NaN
CENDIV,3,Num,Census division,1=New England\n2=Middle Atlantic \n3=East Nor...,NaN
PBA,4,Num,Principal building activity,1=Vacant\n2=Office\n4=Laboratory\n5=Nonrefrige...,NaN
PUBCLIM,5,NUM,Third-party data: ASHRAE climate zone,1=Cold or very cold\n2=Cool\n3=Mixed mild\n4=W...,NaN
...,...,...,...,...,...
ZNGEXP,1245,Num,Imputation flag: Annual natural gas expenditur...,0=Reported\n1=Imputed\n2=Estimated for strip c...,NaN
ZFKCNS,1246,Num,Imputation flag: Annual fuel oil consumption (...,0=Reported\n1=Imputed\n2=Estimated for strip c...,NaN
ZFKEXP,1247,Num,Imputation flag: Annual fuel oil expenditures ($),0=Reported\n1=Imputed\n2=Estimated for strip c...,NaN


You can therefore access the human-readable names of columns like this:

In [16]:
codebook.loc[mdh.CENDIV, cdh.label]

'Census division'

`CENDIV` is the Census Division for each facility in the microdata (remember, every row of the microdata represents a single facility). However, we notice the `CENDIV` column is a list of integers:

In [17]:
microdata[mdh.CENDIV]

0       5
1       9
2       5
3       7
4       2
       ..
6431    8
6432    7
6433    5
6434    5
6435    2
Name: CENDIV, Length: 6436, dtype: int64

We can once again use the codebook to get the "human-readable" Census Division:

In [18]:
codebook.loc[mdh.CENDIV, cdh.codes]

'1=New England\n2=Middle Atlantic  \n3=East North Central        \n4=West North Central \n5=South Atlantic \n6=East South Central \n7=West South Central \n8=Mountain \n9=Pacific'

However, the way that the `codes` are formatted in the CBECS codebook (as a string) is not very helpful for automated analysis. We can transform these strings into a more helpful format, like a Python dictionary. We've included this parsing functionality in the `pycbecs` helper module. Feel free to look through/edit the code there to get it into your desired format.

In [48]:
# We generally don't need to load the codebook again
# I'm just putting this here because of the flow of this blogpost
# If someone runs this cell multiple times, we want them to start with a fresh
# codebook before transforming.
codebook = handler.load_codebook("data/2018microdata_codebook.xlsx")
codebook = handler.transform_codes_to_dict(codebook)
codebook.loc[mdh.CENDIV, cdh.codes]

{1: 'New England',
 2: 'Middle Atlantic',
 3: 'East North Central',
 4: 'West North Central',
 5: 'South Atlantic',
 6: 'East South Central',
 7: 'West South Central',
 8: 'Mountain',
 9: 'Pacific'}

# Excel Example 1
**Description**: Calculate the number of commercial buildings that are 5,000 square feet or smaller

**Excel Methodology**: You can estimate the total number of buildings using the sum of FINALWT for a specified subset
of building records within the CBECS data file. For this example, filter the file for all building
records where the square footage category is 1,001 to 5,000 square feet (SQFTC=2). The file
contains 1,006 such building records. By adding the FINALWT column for these building records,
the result is that the estimated number of buildings that are 5,000 square feet or smaller is
2,832,884.62 (or 2,833 thousand buildings as reported in Table B1). This amount equals 48% of
all commercial buildings, or 2,832,885 ÷ 5,918,212 (the sum of FINALWT for all buildings in the
CBECS reporting sample).

**Python Equivalent**:

In [57]:
# Filter records
sqftc_code = 2
sqft_mask = microdata[mdh.SQFTC] == sqftc_code
filtered_records = microdata[sqft_mask]
number_of_records = len(filtered_records)

# Sum weights to get total number of buildings
# Remember, FINALWT is the "weight" that specifies how many
# buildings an individual sample represents (i.e. row of microdata)
number_of_buildings = filtered_records[mdh.FINALWT].sum()

# Calculate Percent of Total Number of Buildings
perc_of_all_buildings = number_of_buildings / microdata[mdh.FINALWT].sum()

sqftc_codes = codebook.loc[mdh.SQFTC, cdh.codes]
print(f"Number of records: {number_of_records}")
print(f"Number of buildings: {number_of_buildings:,.2f}")
print(f"% of Buildings {sqftc_codes[sqftc_code]}: {perc_of_all_buildings:.0%}")

Number of records: 1006
Number of buildings: 2,832,884.62
% of Buildings 1,001 to 5,000 square feet: 48%


**Python extra**: But the beauty of using a programming language like Python over Excel is that we can easily replicate repetitive,"manual" workflows for a lot of different cases. Using a `pandas` `groupby` statement, we can find the number of buildings for each square footage category with one line of code:

In [72]:
num_builds_per_sqftc = microdata.groupby(mdh.SQFTC)[mdh.FINALWT].sum()

And use the `codebook` to get human-readable category names:

In [73]:
num_builds_per_sqftc.rename(index=codebook.loc[mdh.SQFTC, cdh.codes])

SQFTC
1,001 to 5,000 square feet          2832884.62
5,001 to 10,000 square feet         1359072.45
10,001 to 25,000 square feet         980905.39
25,001 to 50,000 square feet         386085.63
50,001 to 100,000 square feet        217643.34
100,001 to 200,000 square feet        92772.05
200,001 to 500,000 square feet        40071.10
500,001 to 1 million square feet       6664.82
Over 1 million square feet             2111.82
Name: FINALWT, dtype: float64

# Excel Example 2
**Description**: Calculate the total square footage of commercial buildings in the Northeast
Census region

**Excel Methodology**: To calculate total square footage, create a new column for weighted square footage by
multiplying SQFT by FINALWT for each building record. Then filter the file for the Northeast
(REGION=’1’). There are 1,136 such building records. Sum the new weighted square footage
column (for the Northeast only) to get 15,986,079,825 square feet (or 15,986 million square feet
as reported in Table B1).

**Python Equivalent**:

In [78]:
# Filter records
region_code = 1
region_mask = microdata[mdh.REGION] == region_code
filtered_records = microdata[region_mask]
number_of_records = len(filtered_records)

# Calculate "weighted" sq footage
tot_sqft_in_northeast = (filtered_records[mdh.FINALWT]
                         .multiply(filtered_records[mdh.SQFT])
                         .sum()
                        )

region_codes = codebook.loc[mdh.REGION, cdh.codes]
print(f"Number of records: {number_of_records}")
print(f"Total (Weighted) Square Footage in {region_codes[region_code]}: {tot_sqft_in_northeast:,.0f}")

Number of records: 1136
Total (Weighted) Square Footage in Northeast: 15,986,079,825


# Excel Example 3
**Description**: Calculate the weighted average size (mean square feet per building) of
office buildings

**Excel Methodology**: To calculate total square footage, create a new column for weighted square footage by
multiplying SQFT by FINALWT for each building record. Then filter the file for office buildings
(PBA=2). There are 1,329 such building records. Sum the new weighted square footage column
(for office buildings only) to get 16,578,416,219 square feet. Sum FINALWT for office buildings to
get 970,302.56 (estimate for the number of office buildings). To calculate mean square feet for
office buildings, divide the weighted square footage total by the estimated number of office
buildings. The result is 17,085.82 square feet (or 17.1 thousand as reported in Table B1).

**Python Equivalent**:

In [89]:
# Filter records
pba_code = 2
pba_mask = microdata[mdh.PBA] == pba_code
filtered_records = microdata[pba_mask]
number_of_records = len(filtered_records)

# Sum weights to get total number of buildings
# Remember, FINALWT is the "weight" that specifies how many
# buildings an individual sample represents (i.e. row of microdata)
tot_sqft_offices = (filtered_records[mdh.FINALWT]
                    .multiply(filtered_records[mdh.SQFT])
                    .sum()
                   )
weighted_avg_sqft = tot_sqft_offices / filtered_records[mdh.FINALWT].sum()
pba_codes = codebook.loc[mdh.PBA, cdh.codes]
print(f"Number of records: {number_of_records}")
print(f"Total (Weighted) Sq Ft of '{pba_codes[pba_code]}' buildings: {tot_sqft_offices:,.0f}")
print(f"Weighted Avg Square Footage of '{pba_codes[pba_code]}' buildings: {weighted_avg_sqft:,.2f}")

Number of records: 1329
Total (Weighted) Sq Ft of 'Office' buildings: 16,578,416,219
Weighted Avg Square Footage of 'Office' buildings: 17,085.82


# TODO: Weighted Averages are interesting, but the weighted descriptive statistics (e.g.  can be more interesting